In [2]:
import sys; sys.path.append('..')
import numpy as np, elastic_rods
from bending_validation import suppress_stdout
from linkage_vis import LinkageViewer
linkage_name = '20190603_152106_meshID_790c7704-7379-4713-a7c4-c190e4060571'
l = elastic_rods.RodLinkage('../../examples/florin/{}.obj'.format(linkage_name), 8)
#driver=166
driver=l.centralJoint()

#mat = elastic_rods.RodMaterial('+', 2000, 0.3, [5, 5, 0.7, 0.7], stiffAxis=elastic_rods.StiffAxis.D1)
mat = elastic_rods.RodMaterial('rectangle', 20000, 0.3, [8, 12])
l.setMaterial(mat)

with suppress_stdout(): elastic_rods.restlen_solve(l)
jdo = l.dofOffsetForJoint(driver)
fixedVars = list(range(jdo, jdo + 6)) # fix rigid motion for a single joint
fixedVars.append(jdo + 6) # constrain angle at the driving joint
with suppress_stdout(): elastic_rods.compute_equilibrium(l, fixedVars=fixedVars)

view = LinkageViewer(l, width=1024)
#view.setCameraParams(((2.656451698624758, 1.5087349593639046, 0.23168476156681694),
# (0.052583904614345374, 0.026224725204501566, 0.9982721055721517),
# (0.13680349418553736, -0.13680349418553736, -0.8425278443781967)))
view.show()

Renderer(camera=PerspectiveCamera(aspect=2.0, children=(PointLight(color='white', intensity=0.6, position=(0.0…

In [5]:
l.saveVisualizationGeometry('deployed_{}.obj'.format(linkage_name))
#l.writeLinkageDebugData('')

In [17]:
from write_render_files import writeRenderFiles, writeActuators
writeRenderFiles(l, 'Pavilion21', 'pav21')
#writeActuators(l, np.where(torques > 1e-4)[0], directory='AsymPointyDataFlat', name='asymPointy_Fab')

In [10]:
from linkage_utils import writeRodSegments
writeRodSegments(l,'rod_segments_{}.txt'.format(linkage_name), zeroBasedIndexing=True)
np.savetxt('restlen_{}.txt'.format(linkage_name),l.getPerSegmentRestLength())
np.savetxt('normals_{}.txt'.format(linkage_name), np.array([j.normal for j in l.joints()]))
l.writeLinkageDebugData('deployed_{}.msh'.format(linkage_name))

In [41]:
# Compute maximum "stress" appearing at any rod vertex in the structure
max([max(segmentStress) for segmentStress in l.rodStresses()[1]])

24.37145054902055

In [ ]:
# Output actuation torque
for forceScale in np.linspace(1.0, 0, 51):
    name = 'Pavilion14_{}'.format(forceScale)
    with suppress_stdout(): elastic_rods.compute_equilibrium(l, forceScale * externalForces, fixedVars=fixedVars)
    view.update(preserveExisting=True)
    writeRenderFiles(l, directory, name)
    writeActuators(l, np.where(torques > 0.1)[0], directory, name)

In [44]:
# Compute average "stress" appearing over all rod vertices in the structure
np.mean(sum(l.rodStresses()[1], []))

11.81760497978405

In [3]:
from open_linkage import open_linkage
def equilibriumSolver(tgtAngle, l, opts, fv):
    opts.beta = 1e-8
    opts.gradTol = 1e-4
    opts.useIdentityMetric = False
    return elastic_rods.compute_equilibrium(l, tgtAngle, options=opts, fixedVars=fv)
# open_linkage(l, driver, 2 * np.pi/3, 25, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=False);
open_linkage(l, driver, np.deg2rad(110) - l.averageJointAngle, 50, view, zPerturbationEpsilon=0, equilibriumSolver=equilibriumSolver, maxNewtonIterationsIntermediate=20, verbose=10, useTargetAngleConstraint=True);

target angle:  1.1197401311115107
0	1.07334e+06	91608	91608	1	0
10	765102	2186.46	2186.46	1	0
20	763516	892.154	892.154	1	0
2.9783043175556005	763516.2978741741	763516.2978741741	687460.6748610358	75414.83240810737	640.7906050301449
target angle:  1.136069152460128
0	771359	18563.3	18563.3	1	0
10	763378	1205.52	1205.52	1	0
20	762083	1923.56	1923.56	1	0
3.3199329521683105	762083.0308869592	762083.030886959	685807.2933362339	75605.12523811465	670.6123126109907
target angle:  1.1523981738087454
0	765416	7649.93	7649.93	1	0
10	763020	190.183	190.183	0.25	0
20	762144	1285.61	1285.61	0.5	0
3.4028761293603744	762144.0456594746	762144.0456594746	684629.2355198129	76942.48719346941	572.3229461919009
target angle:  1.1687271951573628
0	766034	6488.24	6488.24	1	0
10	764507	857.88	857.88	1	0
20	763935	832.611	832.611	1	0
3.4466716584157573	763934.8750464732	763934.8750464733	684483.2202573617	78935.80009420401	515.8546949072064
target angle:  1.1850562165059801
0	769030	6901.92	6901.92	1	0
10	7676